In [1]:
import numpy as np
import pandas as pd
import json

# Create dictionary with the anwers descriptions

In [2]:
IVS_variables = pd.read_csv("input/IVS_Variable_List.csv")

In [3]:
IVS_variables

,Included,Name,Label,Last EVS,Last WVS,Categories
0,1,A001,Important in life: Family,EVS 2008,WVS6 (2010-2012),1:Very important\n2:Rather important\n3:Not ve...
1,1,A002,Important in life: Friends,EVS 2008,WVS6 (2010-2012),1:Very important\n2:Rather important\n3:Not ve...
2,0,A003,Important in life: Leisure time,EVS 2008,WVS6 (2010-2012),-5:Missing; Unknown\n-4:Not asked in survey\n-...
3,0,A004,Important in life: Politics,EVS 2008,WVS6 (2010-2012),1:Very important\n2:Rather important\n3:Not ve...
4,1,A005,Important in life: Work,EVS 2008,WVS6 (2010-2012),1:Very important\n2:Rather important\n3:Not ve...
...,...,...,...,...,...,...
1422,0,w007,Spouse/partner had/has how many employees,EVS 2008,NaN,1:None\n2:1-9\n3:10-24\n4:25 or more\n-5:Missi...
1423,0,w008,Does spouse/partner supervise someone,EVS 2008,NaN,0:No\n1:Yes\n-5:Missing; Unknown\n-4:Not asked...
1424,0,w009,How many people does she/he supervise,EVS 2008,NaN,1:1-9\n2:10-24\n3:25 or more\n-5:Missing; Unkn...
1425,0,w010,Spouse/partner experienced unemployment longer...,EVS 2008,NaN,0:No\n1:Yes\n-5:Missing; Unknown\n-4:Not asked...


In [4]:
IVS_variables = IVS_variables.loc[IVS_variables.Included == 1].copy()
IVS_variables.drop(["Included", "Last EVS", "Last WVS"], axis=1, inplace=True)

In [5]:
cat_dict_column = []

for idx, row in IVS_variables.iterrows():
    codes = {}
    for line in row.Categories.split("\n")[:-1]: # there is always a \n at the end
        try:
            key = line.split(":")[0]
            value = line.split(":")[1]
            codes[key] = value
        except:
            continue
    cat_dict_column.append(codes)

In [6]:
IVS_variables["Categories"] = cat_dict_column

In [7]:
IVS_variables.set_index("Name", inplace=True)

In [8]:
variables_dict = IVS_variables.to_dict(orient="index")

## We need to pop some keys
ZMEN_010 is not present in the dataset

the following ones are the values we aggregated on

S002
S002EVS
S001
S017

X048WVS
X048
x048a_n1
x048b_n2
x048c_n3

In [9]:
len(variables_dict.keys())

87

In [10]:
to_drop = ["ZMEN_010","S001", "S002", "S002EVS", "S003", "S017",
          "X048WVS", "X048", "x048a_n1", "x048b_n2", "x048c_n3"]

for key in to_drop:
    variables_dict.pop(key)

len(variables_dict.keys())

76

# Country data

In [11]:
# load the list of GID at the country level (admn_0)
country_gid = pd.read_csv("gid/country_gid.csv")
country_gid

,country,countrycode,countrycode_alpha2,adm_level,adm_area_1,adm_area_1_code,adm_area_2,adm_area_2_code,adm_area_3,adm_area_3_code,gid
0,Aruba,ABW,AW,0,NaN,NaN,NaN,NaN,NaN,NaN,ABW
1,Anguilla,AIA,AI,0,NaN,NaN,NaN,NaN,NaN,NaN,AIA
2,Palestina,PSE,PS,0,NaN,NaN,NaN,NaN,NaN,NaN,PSE
3,Cyprus,CYP,CY,0,NaN,NaN,NaN,NaN,NaN,NaN,CYP
4,Micronesia,FSM,FM,0,NaN,NaN,NaN,NaN,NaN,NaN,FSM
...,...,...,...,...,...,...,...,...,...,...,...
251,Paracel Islands,XPI,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,XPI
252,Yemen,YEM,YE,0,NaN,NaN,NaN,NaN,NaN,NaN,YEM
253,Zambia,ZMB,ZM,0,NaN,NaN,NaN,NaN,NaN,NaN,ZMB
254,South Africa,ZAF,ZA,0,NaN,NaN,NaN,NaN,NaN,NaN,ZAF


In [34]:
# load the IVS data agrgegated by country
IVS_country = pd.read_csv("out/IVS_grouped_by_country.csv", encoding="utf-16")
IVS_country

,Survey,Wave,Country Numeric,Country Name,Country ISO3,Sample Size,A001_-5,A001_-4,A001_-2,A001_-1,...,X053_1,X053_2,X053_3,X053_4,X053_5,X053_6,X053_7,X053_8,X053_9,X053_10
0,EVS,1981-1984,56,Belgium,BEL,1145,0.0,1.0,0.000000,0.000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,EVS,1981-1984,124,Canada,CAN,1254,0.0,1.0,0.000000,0.000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,EVS,1981-1984,208,Denmark,DNK,1182,0.0,1.0,0.000000,0.000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,EVS,1981-1984,250,France,FRA,1200,0.0,1.0,0.000000,0.000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,EVS,1981-1984,276,Germany,DEU,1305,0.0,1.0,0.000000,0.000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,WVS,2010-2014,818,Egypt,EGY,1523,0.0,0.0,0.000000,0.000,...,0.080814,0.049620,0.073058,0.045724,0.052922,0.037831,0.046540,0.063399,0.017063,0.032774
362,WVS,2010-2014,840,United States of America (the),USA,2232,0.0,0.0,0.003548,0.000,...,0.106743,0.043508,0.054001,0.043248,0.152357,0.101210,0.100090,0.122997,0.110341,0.136770
363,WVS,2010-2014,858,Uruguay,URY,1000,0.0,0.0,0.000000,0.001,...,0.329000,0.104000,0.062000,0.057000,0.130000,0.037000,0.040000,0.065000,0.045000,0.064000
364,WVS,2010-2014,860,Uzbekistan,UZB,1500,0.0,0.0,0.000000,0.000,...,0.237333,0.094667,0.078667,0.075333,0.078667,0.061333,0.031333,0.058000,0.042000,0.094667


In [13]:
IVS_country[~ IVS_country["Country ISO3"].isin(country_gid.countrycode)]

,Survey,Wave,Country Numeric,Country Name,Country ISO3,Sample Size,A001_-5,A001_-4,A001_-2,A001_-1,...,X053_1,X053_2,X053_3,X053_4,X053_5,X053_6,X053_7,X053_8,X053_9,X053_10
15,EVS,1981-1984,909,North Ireland,Nan,312,0.0,1.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44,EVS,1990-1993,909,North Ireland,Nan,304,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
77,EVS,1999-2001,909,North Ireland,Nan,1000,0.0,0.0,0.003159,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
87,EVS,2008-2010,197,Northern Cyprus,NaN,500,0.0,0.0,0.007390,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
122,EVS,2008-2010,909,North Ireland,Nan,500,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
123,EVS,2008-2010,915,Kosovo,NaN,1601,0.0,0.0,0.008117,0.001159,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
160,WVS,1994-1998,101,Republika Srpska,NaN,400,0.0,0.0,0.000000,0.005000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
207,WVS,1994-1998,914,Bosnia,NaN,800,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
IVS_country_has_gid = IVS_country[IVS_country["Country ISO3"].isin(country_gid.countrycode)].copy()

In [15]:
IVS_country_has_gid['json'] = IVS_country_has_gid.apply(lambda x: x.to_dict(), axis=1)

In [16]:
# now we have a flat dict, we will need to organize it
IVS_country_has_gid.json[0]

{'Survey': 'EVS',
 'Wave': '1981-1984',
 'Country Numeric': 56,
 'Country Name': 'Belgium',
 'Country ISO3': 'BEL',
 'Sample Size': 1145,
 'A001_-5': 0.0,
 'A001_-4': 1.0000000024896332,
 'A001_-2': 0.0,
 'A001_-1': 0.0,
 'A001_1': 0.0,
 'A001_2': 0.0,
 'A001_3': 0.0,
 'A001_4': 0.0,
 'A002_-5': 0.0,
 'A002_-4': 1.0000000024896332,
 'A002_-2': 0.0,
 'A002_-1': 0.0,
 'A002_1': 0.0,
 'A002_2': 0.0,
 'A002_3': 0.0,
 'A002_4': 0.0,
 'A005_-5': 0.0,
 'A005_-4': 1.0000000024896332,
 'A005_-3': 0.0,
 'A005_-2': 0.0,
 'A005_-1': 0.0,
 'A005_1': 0.0,
 'A005_2': 0.0,
 'A005_3': 0.0,
 'A005_4': 0.0,
 'A009_-5': 0.0,
 'A009_-4': 0.0,
 'A009_-2': 0.0,
 'A009_-1': 0.01220362320486207,
 'A009_1': 0.30505301707231897,
 'A009_2': 0.3836794675948685,
 'A009_3': 0.2494125320035641,
 'A009_4': 0.040209933866843454,
 'A009_5': 0.009441428747183164,
 'A029_-5': 0.0,
 'A029_-4': 0.0,
 'A029_-2': 0.0,
 'A029_-1': 0.0,
 'A029_0': 0.8024454438528174,
 'A029_1': 0.1975545586368192,
 'A032_-5': 0.0,
 'A032_-4': 0

In [17]:
columns = IVS_country_has_gid.columns

properties = []

for idx, row in IVS_country_has_gid.iterrows():
    print(idx)
    new_dict = {}
    
    # for each row we create a dict
    # {..., 
    #  column_id: {label: 'description',
    #              frequencies: {dict with the anwers onehot encoded}
    #.             categories: {dict with the answers codes}}, 
    #  ...}
    
    # label and categories are from variables_dict
    
    for answer_code in variables_dict.keys():
        
        # we already have label and categories
        sub_dict = variables_dict[answer_code]
        
        # we need frequencies
        # in the row json we have a flat dict
        # we want to aggregate by name of the answer
        sub_dict["Frequencies"] = {}
        
        for key, item in row.json.items():
            if key.startswith(answer_code):
                sub_dict["Frequencies"][key] = item
        new_dict[answer_code] = sub_dict
        
    properties.append(new_dict)    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
78
79
80
81
82
83
84
85
86
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
28

In [18]:
df_to_save = IVS_country_has_gid[ ['Survey', 'Wave', 'Country Numeric', 
                           'Country Name', 'Country ISO3', 'Sample Size'] ].copy()

In [19]:
df_to_save["GID"] = IVS_country_has_gid['Country ISO3']
df_to_save["Properties"] = properties

In [20]:
df_to_save

,Survey,Wave,Country Numeric,Country Name,Country ISO3,Sample Size,GID,Properties
0,EVS,1981-1984,56,Belgium,BEL,1145,BEL,{'A001': {'Label': 'Important in life: Family'...
1,EVS,1981-1984,124,Canada,CAN,1254,CAN,{'A001': {'Label': 'Important in life: Family'...
2,EVS,1981-1984,208,Denmark,DNK,1182,DNK,{'A001': {'Label': 'Important in life: Family'...
3,EVS,1981-1984,250,France,FRA,1200,FRA,{'A001': {'Label': 'Important in life: Family'...
4,EVS,1981-1984,276,Germany,DEU,1305,DEU,{'A001': {'Label': 'Important in life: Family'...
...,...,...,...,...,...,...,...,...
361,WVS,2010-2014,818,Egypt,EGY,1523,EGY,{'A001': {'Label': 'Important in life: Family'...
362,WVS,2010-2014,840,United States of America (the),USA,2232,USA,{'A001': {'Label': 'Important in life: Family'...
363,WVS,2010-2014,858,Uruguay,URY,1000,URY,{'A001': {'Label': 'Important in life: Family'...
364,WVS,2010-2014,860,Uzbekistan,UZB,1500,UZB,{'A001': {'Label': 'Important in life: Family'...


In [21]:
df_to_save.to_csv("to_upload/IVS_country_GID.csv", index=False)

# UPLOAD
## Loads the data from csv, there is no need to run the cells above

In [22]:
import pandas
import psycopg2

In [31]:
def create_IVS_country_table(cur):
    sql_create_country_stats_table = """ 
        CREATE TABLE IF NOT EXISTS IVS_country (
            survey text NOT NULL,
            wave text NOT NULL,
            country text NOT NULL,
            countrycode text NOT NULL,
            sample_size integer NOT NULL,
            gid text NOT NULL,
            properties jsonb,
            UNIQUE (survey, wave, country, countrycode),
            PRIMARY KEY (survey, wave, country, countrycode)
        )"""

    cur.execute(sql_create_country_stats_table)

    print("IVS_country table created")

In [24]:
def drop_IVS_country_table(cur):
    sql_drop_country_stats_table = """DROP TABLE IVS_country"""
    
    try:
        cur.execute(sql_drop_country_stats_table)
        conn.commit() 
        print("IVS_country table deleted")
    except:
        print("IVS_country not present")

In [28]:
def import_IVS_country_data(cur,PATH):
    
    INSERT_STATEMENT = 'INSERT INTO IVS_country \
                            (survey, wave, \
                             country, countrycode,  \
                             sample_size, gid, \
                             properties) \
                        VALUES (%s, %s, \
                                %s, %s, \
                                %s, %s, \
                                %s);'
    
    data = pd.read_csv(PATH)

    for idx, row in data.iterrows():
        print(idx)
        cur.execute(INSERT_STATEMENT, (row['Survey'], row['Wave'], 
                                       row['Country Name'], row['Country ISO3'], 
                                       row['Sample Size'], row['GID'],
                                       json.dumps(row["Properties"])))

In [32]:
# Connect to covid19db.org
conn = psycopg2.connect(
    host='covid19db.org',
    port=5432,
    dbname='covid19db_adm_play',
    user='covid19db_adm_rw',
    password='fGt962FdeG2yXj3c4d3'
)

cur = conn.cursor()

drop_IVS_country_table(cur)
conn.commit()


create_IVS_country_table(cur)
conn.commit()

# there are 358 rows 
import_IVS_country_data(cur, "to_upload/IVS_country_GID.csv")
conn.commit()


IVS_country not present
IVS_country table created
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264


# TEST

In [33]:
cur.execute("""SELECT * FROM IVS_country""")

data = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])
data


,survey,wave,country,countrycode,sample_size,gid,properties
0,EVS,1981-1984,Belgium,BEL,1145,BEL,{'A001': {'Label': 'Important in life: Family'...
1,EVS,1981-1984,Canada,CAN,1254,CAN,{'A001': {'Label': 'Important in life: Family'...
2,EVS,1981-1984,Denmark,DNK,1182,DNK,{'A001': {'Label': 'Important in life: Family'...
3,EVS,1981-1984,France,FRA,1200,FRA,{'A001': {'Label': 'Important in life: Family'...
4,EVS,1981-1984,Germany,DEU,1305,DEU,{'A001': {'Label': 'Important in life: Family'...
...,...,...,...,...,...,...,...
353,WVS,2010-2014,Egypt,EGY,1523,EGY,{'A001': {'Label': 'Important in life: Family'...
354,WVS,2010-2014,United States of America (the),USA,2232,USA,{'A001': {'Label': 'Important in life: Family'...
355,WVS,2010-2014,Uruguay,URY,1000,URY,{'A001': {'Label': 'Important in life: Family'...
356,WVS,2010-2014,Uzbekistan,UZB,1500,UZB,{'A001': {'Label': 'Important in life: Family'...
